# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# The necessary packages are imported here
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking the current working directory
print(os.getcwd())

# Getting the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Creating a "for loop" to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# joinning the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/Users/abbas/Documents/udacity/data engineering nano/Data_Modelling/data_modelling_with_cassandra_project
['/Users/abbas/Documents/udacity/data engineering nano/Data_Modelling/data_modelling_with_cassandra_project/event_data/2018-11-15-events.csv', '/Users/abbas/Documents/udacity/data engineering nano/Data_Modelling/data_modelling_with_cassandra_project/event_data/2018-11-22-events.csv', '/Users/abbas/Documents/udacity/data engineering nano/Data_Modelling/data_modelling_with_cassandra_project/event_data/2018-11-09-events.csv', '/Users/abbas/Documents/udacity/data engineering nano/Data_Modelling/data_modelling_with_cassandra_project/event_data/2018-11-18-events.csv', '/Users/abbas/Documents/udacity/data engineering nano/Data_Modelling/data_modelling_with_cassandra_project/event_data/2018-11-04-events.csv', '/Users/abbas/Documents/udacity/data engineering nano/Data_Modelling/data_modelling_with_cassandra_project/event_data/2018-11-01-events.csv', '/Users/abbas/Documents/udacity/data engi

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))
        


8056


In [4]:
len(full_data_rows_list)

8056

In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# Making a connection to a Cassandra instance our local machine 
# (127.0.0.1)
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])

    # Establishing connection and beginning executing queries
    session = cluster.connect()
except Exception as e:
    print(e)


#### Create Keyspace

In [7]:
# Here, we create a keyspace (database), sparkify_cassandra
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify_cassandra
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# we connect to the KEYSPACE specified above
try:
    session.set_keyspace('sparkify_cassandra')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
#Here, we want to retrieve the artist, song title and song's length that was heard during sessionId = 338, and itemInSession = 4 in our database.
#Since cassandra is a query-based database and JOINs are't allowed, we create our tables according to our queries.

#The table, songs_by_user_session is created below. Since we want to retrieve data based on session_id and item_in_session, we start with...
#... both when defining our table, thereby clustering on the two columns. Consequently, our primary and partition keys are session_id and item_in_session
query = "CREATE TABLE IF NOT EXISTS songs_by_user_session"
query = query + "(session_id int, item_in_session int, artist text, song text, length float, PRIMARY KEY(session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

#The query to select the  artist, song title and song's length that was heard during sessionId = 338, and itemInSession = 4
#We can directly replace the %s with 338 and 4, but we intentionally put the values as placeholders here so we can add the values...
#...When executing the queries

select_query = "select artist, song, length FROM songs_by_user_session WHERE session_id = %s AND item_in_session = %s"    

# We set up the CSV file here for reading
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        #Here, we assign the INSERT statements into the `query` variable
        query = "INSERT INTO songs_by_user_session (session_id, item_in_session, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"

        # we populate our table, songs_by_user_session by assigning which column element should be assigned for each column in the INSERT statement.
        #The lines 8,3,0, 9 and 5 are the columns with the name  "sessionId", "ItemInSession", "artist", "song" and "length" in the event_datafile_new.csv file
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

# Here, we confirm if the data was populated successfully into the table by adding  SELECT statement
try:
    rows = session.execute(select_query, (338, 4))
except Exception as e:
    print(e)
    
for row in rows:
    print("Artist: "+row.artist, "\nSong Title: "+row.song, "\nSong Length: "+str(row.length))

Artist: Faithless 
Song Title: Music Matters (Mark Knight Dub) 
Song Length: 495.30731201171875


#### Do a SELECT to verify that the data have been inserted into each table

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [10]:
#Here, we want to retrieve the name of artist, song (sorted by ItemInSession) and user's first and last name for the user_id = 10 and session_id =182 
#We create the table, artist_songs_by_item_session below, putting our query in mind.
#Since we'll be filtering on two columns, "user_id" and "session_id" with the result being sorted by "item_in_session", ther's need to create a 
#composite partition key ((user_id, session_id), item_in_session))
#Notice: THE PRIMARY KEY IS MADE UP OF EITHER JUST THE PARTITION KEY OR MAY ALSO INCLUDE ADDITIONAL CLUSTERING COLUMNS (like the item_in_session)

query = "CREATE TABLE IF NOT EXISTS artist_songs_by_item_session"
query = query + "(user_id int, session_id int,  item_in_session int, artist text, song text, \
                first_name text, last_name text, PRIMARY KEY((user_id, session_id), item_in_session))"                
try:
    session.execute(query)
except Exception as e:
    print(e)

#The query to select the  artist, song, first and last name from the artist_songs_by_item_session for the user_id = 10 and session_id = 182
#We can directly replace the %s with 10 and 182, but we intentionally put the values as placeholders here so we can add the values...
#...When executing the queries
select_query = "select artist, song, first_name, last_name FROM artist_songs_by_item_session WHERE user_id = %s AND session_id = %s"    

# We set up the CSV file here for reading
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        #Here, we assign the INSERT statements into the `query` variable
        query = "INSERT INTO artist_songs_by_item_session (user_id, session_id, item_in_session, artist, song, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"

        # we populate our table, artist_songs_by_item_session by assigning which column element should be assigned for each column in the INSERT statement.
        #The lines 10, 8, 3, 0, 9, 1 and 4  are the columns with the name  "userId", "sessionId", "itemInSession", "song", "firstName" and "lastName" in the event_datafile_new.csv file
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))


# Here, we confirm if the data was populated successfully into the table by adding  SELECT statement
try:
    rows = session.execute(select_query, (10, 182))
except Exception as e:
    print(e)
for row in rows:
    print("Artist: "+row.artist, "\nSong Title: "+row.song, "\nFirst Name: "+row.first_name, "\nLast Name: "+row.last_name)
                    

Artist: Down To The Bone 
Song Title: Keep On Keepin' On 
First Name: Sylvie 
Last Name: Cruz
Artist: Three Drives 
Song Title: Greece 2000 
First Name: Sylvie 
Last Name: Cruz
Artist: Sebastien Tellier 
Song Title: Kilometer 
First Name: Sylvie 
Last Name: Cruz
Artist: Lonnie Gordon 
Song Title: Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 
First Name: Sylvie 
Last Name: Cruz


In [14]:

#Here, we want to retrieve every user's first and last names who listened to the song 'All Hands Against His Own'
#We create the table, user_song_details below, putting our query in mind.
#Since we want to retrieve users who listened to a particular song, there is need to  partition by "song"
#An Assumption is that there might be two or more people with thesame first and last names, we consider ...
#...using user_id as a clustering column to  add a unique contraint because user_ids are technically unique indentifiers
# Additionally, our reason for this is that values for all columns defined in the PARTITION KEY have to be supplied to retrieve data from a table, .
# We ONLY want to use the song with the WHERE clause. So, it makes sense to define our PRIMARY KEY as it is below


query = "CREATE TABLE IF NOT EXISTS user_song_details"
query = query + "(song text, user_id int, first_name text, last_name text,  PRIMARY KEY((song), user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

#The query to select the  first and last names from the user_song_details for those who listened to the song, 'All Hands Against His Own' 
#Unlike the other ones above, we have not made use of any place holder here. we demonstrate the possibility of adding the conditional values directly

select_query = "select first_name, last_name FROM user_song_details WHERE song = 'All Hands Against His Own'"    

# We set up the CSV file here for reading
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        #Here, we assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_song_details (song, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"

        # we populate our table, user_song_details by assigning which column element should be assigned for each column in the INSERT statement.
        #The lines 9, 10, 1 and 4  are the columns with the names "song", "userId", "firstName" and "lastName" in the event_datafile_new.csv file
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))


# Here, we confirm if the data was populated successfully into the table by adding  SELECT statement
try:
    rows = session.execute(select_query)
except Exception as e:
    print(e)
for row in rows:
    print("First Name: "+row.first_name, "\nLast Name: "+row.last_name)

First Name: Jacqueline 
Last Name: Lynch
First Name: Tegan 
Last Name: Levine
First Name: Sara 
Last Name: Johnson


### Drop the tables before closing out the sessions

In [15]:
#Dropping the table before closing out the sessions
session.execute("DROP TABLE IF EXISTS songs_by_user_session")
session.execute("DROP TABLE IF EXISTS artist_songs_by_item_session")
session.execute("DROP TABLE IF EXISTS user_song_details")

### Close the session and cluster connection¶

In [16]:
session.shutdown()
cluster.shutdown()